# 🔎 Análisis exploratorio del dataset Madrid Polution (2001- 2022)

Este proyecto utiliza el dataset “MadridPolution2001-2022.csv”, que contiene registros horarios de contaminantes atmosféricos medidos en la estación Escuelas Aguirre (Madrid) desde enero de 2001 hasta marzo de 2022. El objetivo principal es analizar la evolución de la calidad del aire en Madrid, identificando tendencias, picos y patrones estacionales en los principales contaminantes.

El dataset incluye las siguientes variables:
- **BEN**: Benceno (µg/m³)
- **CH4**: Metano (mg/m³)
- **CO**: Monóxido de carbono (mg/m³)
- **EBE**: Etilbenceno (µg/m³)
- **NMHC**: Hidrocarburos no metánicos (mg/m³)
- **NO**: Monóxido de nitrógeno (µg/m³)
- **NO2**: Dióxido de nitrógeno (µg/m³)
- **NOx**: Óxidos de nitrógeno (µg/m³)
- **O3**: Ozono (µg/m³)
- **PM10**: Partículas < 10 µm (µg/m³)
- **PM25**: Partículas < 2.5 µm (µg/m³)
- **SO2**: Dióxido de azufre (µg/m³)
- **TCH**: Hidrocarburos totales (mg/m³)
- **TOL**: Tolueno (µg/m³)

El análisis se centra en la evolución temporal, la comparación entre contaminantes y la identificación de patrones estacionales y extremos.

## 🌱 Alba (limpieza y calidad de datos)

Convertimos la columna Time, originalmente en formato string, a tipo datetime para asegurar consistencia temporal y permitir análisis por fecha y hora. También creamos nuevas columnas (year, month, day, hour) a partir de la columna de fecha para facilitar el análisis exploratorio por periodos de tiempo.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el nuevo dataset
csv_path = r"data/MadridPolution2001-2022.csv"

# Leer sin parsear fechas inicialmente para detectar el nombre de columna correcto
df = pd.read_csv(csv_path)

# --- LIMPIEZA DE FECHA ---
# La columna "time" la convertimos directamente a datetime
df["Time"] = pd.to_datetime(df["Time"], errors="coerce") # Con errors="coerce"

# Verificamos si se han generado nulos al convertir la columna "Time" a formato datetime
print("Nulos en la columna Time después de la conversión: ", df["Time"].isna().sum())

# Creamos columnas auxiliares para explorar temporalmente el dataset, pero de momento no haremos una limpieza estricta
df["year"] = df["Time"].dt.year
df["month"] = df["Time"].dt.month
df["day"] = df["Time"].dt.day
df["hour"] = df["Time"].dt.hour


# Mostramos los resultados
print("Columnas detectadas en el CSV:", df.columns.tolist())
print("Dimensiones del dataset:", df.shape)
print(df.head(10))

Analizamos la presencia de valores nulos en todas las columnas y calculamos su porcentaje respecto al total de filas, identificando aquellas variables con mayor pérdida de información

Determinamos qué variables presentan problemas de calidad de datos debido a la alta proporción de valores ausentes, lo que afecta la representatividad de las mediciones y debe considerarse en futuros análisis.

Comprobamos la existencia de filas duplicadas en el dataset y evaluamos su impacto, eliminándolas en caso de ser necesario para evitar sesgos en el análisis.

Revisamos los tipos de datos de cada columna para asegurar que las variables numéricas se encuentren en formato correcto (float/int) y no en formato texto, evitando errores en el procesamiento posterior.

Analizamos si existen valores anómalos o fuera de rango en las variables (ej. concentraciones negativas o valores extremadamente altos) para garantizar la coherencia de las mediciones.

## 📊 Robert (análisis general y descriptivo)

- Promedio anual: ¿Cuál es la concentración media de cada contaminante por año? (tabla resumen por año y contaminante).
- Mejora: Visualizar la evolución anual de los principales contaminantes con gráficos de líneas.


- Mes con peor calidad del aire: ¿Qué mes tuvo la media más alta de NO₂ cada año? (gráfico de barras mensual por año).
- Mejora: Mostrar el mes más crítico para cada año y visualizar la tendencia mensual agregada.

- Distribución de O₃: ¿Cuál es el rango de concentraciones de O₃ más frecuente? (histograma global y por año).
- Mejora: Analizar la variabilidad anual y estacional de O₃.

- Picos diarios: ¿Qué día presentó la concentración más alta de PM10 en cada año?
- Mejora: Visualizar los días de picos extremos y analizar si hay patrones estacionales o de eventos puntuales.

## 🌍 David (comparaciones y correlaciones)

- Estacionalidad en un año: ¿Qué contaminante presenta mayores diferencias entre invierno y verano?
- Mejora: Analizar la estacionalidad de todos los contaminantes y visualizar la diferencia entre estaciones.

- Correlación: ¿Existe correlación entre los niveles diarios de NO₂ y CO en el periodo analizado?
- Mejora: Analizar correlaciones entre más contaminantes y visualizar la matriz de correlación.

- Laborables vs fines de semana: ¿Hay diferencias en los niveles de NO₂ promedio entre días laborables y fines de semana?
- Mejora: Analizar diferencias para más contaminantes y visualizar la variación semanal.

- Variación horaria: ¿A qué horas del día se concentran los picos de NO₂ en promedio durante el año? (curva horaria).
- Mejora: Analizar la variación horaria de más contaminantes y comparar entre años.